В облаке развернут сервис, реализованны на flask и представляющий следующие возможности:
* Обучение разных архитектур ml-моделей: поддерживаются линейные и ансамблевые модели (Random Forest и Extra Trees) как для задачи классификации, так и для задачи регрессии. Пользователь самостоятельно специфицирует архитектуру алгоритма с соответствущим типом решаемой задачи.
* Возвращение предсказаний по моделям.
* Возвращение списка доступных для обучения классов моделей.
* Обучение заново и удаление уже обученных моделей.


In [29]:
SERV_URL = 'http://nataliav.pythonanywhere.com/'
# Для создания web-приложения используется pythonanywhere.com

In [2]:
import requests

#### Проверка работы удалённого сервиса

In [3]:
r = requests.get(SERV_URL+'test')
r.text

"app is run! Server's time is [2021-10-27 23:39:08.807255]"

# Обзор доступных классов моделей

In [4]:
r = requests.get(SERV_URL+'classes')
r.json()

['RandomForestClassifier',
 'RandomForestRegressor',
 'ExtraTreesClassifier',
 'ExtraTreesRegressor',
 'LogReg',
 'LinReg']

# Обзор уже имеющихся моделей

In [5]:
r = requests.get(SERV_URL+'models')
r.json()

[{'hyper_params': {'random_state': 0},
  'is_fitted': False,
  'name': 'default_model_for_tests',
  'type': 'RandomForestClassifier',
  'uid': 1}]

# Создание новой модели

In [6]:
r = requests.post(SERV_URL+'models', json={'type': 'LogReg'})
ans = r.json()
my_current_model_id = ans['uid']
print(ans)

{'hyper_params': {'random_state': 0}, 'is_fitted': False, 'name': 'default_model_for_tests', 'type': 'LogReg', 'uid': 2}


#### Забыли указать имя и ГП - обновляем параметры (для удобства)

In [8]:
r = requests.put(SERV_URL+'models/15', json={'name':'my_log_reg_clf', 'hyper_params':{'random_state': 42}})
r.json()

###### В случае успеха возвращается новое описание модели. Описания нет, так как мы ошиблись и указали несущетсвующий UID. Попробуем ещё раз

In [9]:
r = requests.put(SERV_URL+'models/'+str(my_current_model_id), json={'name':'my_log_reg_clf', 'hyper_params':{'random_state': 42}})
r.json()

{'hyper_params': {'random_state': 42},
 'is_fitted': False,
 'name': 'my_log_reg_clf',
 'type': 'LogReg',
 'uid': 2}

# Ещё раз взглянем на список существующих моделей

In [11]:
r = requests.get(SERV_URL+'models')
r.json()

[{'hyper_params': {'random_state': 0},
  'is_fitted': False,
  'name': 'default_model_for_tests',
  'type': 'RandomForestClassifier',
  'uid': 1},
 {'hyper_params': {'random_state': 42},
  'is_fitted': False,
  'name': 'my_log_reg_clf',
  'type': 'LogReg',
  'uid': 2}]

# Удалим ненужную модель

In [12]:
r = requests.delete(SERV_URL+'models/1')
print('Удаление успешно?', r.text)
r = requests.get(SERV_URL+'models')
r.json()

Удаление успешно? "True"



[{'hyper_params': {'random_state': 42},
  'is_fitted': False,
  'name': 'my_log_reg_clf',
  'type': 'LogReg',
  'uid': 2}]

# Обучим модель

In [13]:
import numpy as np

#### Возьмём тестовый датасет

In [14]:
from sklearn.datasets import load_iris

In [15]:
X, y = load_iris(return_X_y=True)

In [16]:
data = {
    'x':[[float(v) for v in row] for row in X],
    'y':[int(v) for v in y]
}

#### Обучение

In [17]:
r = requests.put(SERV_URL+'models/15/fit', json=data)
r.text

'Wrong model`s UID (15)'

###### ой, ошиблись с uid модели

In [18]:
r = requests.put(SERV_URL+'models/'+str(my_current_model_id)+'/fit', json=data)
r.text

'fitted model with uid 2'

#### Посмотрим состояние моделей

In [19]:
r = requests.get(SERV_URL+'models')
r.json()

[{'hyper_params': {'random_state': 42},
  'is_fitted': True,
  'name': 'my_log_reg_clf',
  'type': 'LogReg',
  'uid': 2}]

# Применим классификатор

#### Возьмём всё ту же обучающую выборку

In [20]:
X, y = load_iris(return_X_y=True)

In [21]:
data = {
    'x':[[float(v) for v in row] for row in X],
    'proba':int(True)
}

#### Прогноз

In [22]:
r = requests.post(SERV_URL+'models/'+str(my_current_model_id)+'/predict', json=data)

In [23]:
ans = r.json()
preds = np.array(ans['x']).reshape(ans['shape'])
preds[:5]

array([[9.81814878e-01, 1.81851076e-02, 1.43958387e-08],
       [9.71755199e-01, 2.82447708e-02, 3.01071602e-08],
       [9.85463201e-01, 1.45367871e-02, 1.23028833e-08],
       [9.76312180e-01, 2.36877799e-02, 3.96557410e-08],
       [9.85396842e-01, 1.46031465e-02, 1.19200048e-08]])

#### Оценим качество

In [24]:
from sklearn.metrics import accuracy_score

In [25]:
data = {
    'x':[[float(v) for v in row] for row in X],
    'proba':int(False)
}

In [26]:
r = requests.post(SERV_URL+'models/'+str(my_current_model_id)+'/predict', json=data)

In [27]:
ans = r.json()
preds = np.array(ans['x']).reshape(ans['shape'])
preds[:5]

array([0., 0., 0., 0., 0.])

In [28]:
accuracy_score(y, preds)

0.9733333333333334